VISIBILITY DISTANCE PREDICTION

PROBLEM STATEMENT:
Air Traffic Control (ATC) requires accurate tracking of weather conditions to predict visibility distance, which is a critical parameter for safe flight operations. Since the ability to fly planes heavily depends on sufficient visibility, developing a reliable prediction model for visibility distance is essential to ensure safety and efficiency in aviation.

DATA COLLECTION:
The dataset was collected from public meteorological repositories, government open-data platforms, aviation weather stations, and Kaggle weather datasets, which provide hourly weather parameters like temperature, humidity, visibility, wind speed, and pressure. The raw data was merged, cleaned, and standardized while retaining missing values and outliers to reflect real-world conditions.

FEATURE INFORMATION:
DATE
VISIBILITY - Distance from which can object can be seen.
DRYBULBTEMPF - Dry bulb temperature (degrees Farenheit). Most commonly reported standard temperature.
WETBULBTEMPF - Wet bulb temperature (degrees Farenheit)
DewPointTempF - Dew point temperature (degrees Farenheit)
Relative Humidity - Relative humidity (percent)
WindSpeed - Wind speed (miles per hour)
Wind Direction - Wind direction from true north using compass directions.
StationPressure - - Atmospheric pressure (inches of Mercury, or "in Hg").
SeaLevelPressure - Sea level pressure (in Hg).
Precip - Total precipitation in the past hour (in inches)

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

df=pd.read_csv(r'Visibility_weather_data.csv')